Họ tên: Nguyễn Huỳnh Hải Đăng \
MSSV: 20521159

# Load libarary and data

In [1]:
import matplotlib.pyplot as plt
from geopandas.tools import sjoin
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from sklearn.cluster import KMeans
import os
%matplotlib inline

In [2]:
path_data="../Lab_2_3/data/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp"
path_data_click="./data/click_data.json"

In [3]:
os.path.exists(path_data)

True

In [4]:
os.listdir("../Lab_2_3/data/CSL_HCMC")

['.git',
 '.gitignore',
 '.gitmodules',
 '.ipynb_checkpoints',
 'Data',
 'data_analysis',
 'index.html',
 'modules',
 'outputs',
 'README.md',
 'scenarios',
 'Software_VN']

In [5]:
data = gpd.read_file(path_data)

In [6]:
data = data.to_crs(epsg=4326)

In [7]:
data_new = data[["Dist_ID","Dist_Name","Shape_Leng","Shape_Area","geometry"]]
data_new = data_new.to_crs(epsg=4326)
#data_new.head()

In [8]:
data_click = gpd.read_file(path_data_click)

In [9]:
data4heatmap = data[["Dist_ID","Dist_Name","Den_2017","Den_2019","Shape_Leng","Shape_Area","geometry"]]

In [10]:
data4heatmap["Rate_den_grow"] = data4heatmap["Den_2019"]/data4heatmap["Den_2017"]

C:\Users\Dang\anaconda3\envs\CS116\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [11]:
data4heatmap_5 = data4heatmap.sort_values("Rate_den_grow").tail()[["Dist_ID","Dist_Name","Shape_Leng","Shape_Area","geometry"]]

In [12]:
data_dist_click = sjoin(data_click,data4heatmap_5)

In [13]:
data_dist_click["x"]= data_dist_click.geometry.x
data_dist_click["y"]= data_dist_click.geometry.y

In [14]:
list_points=[]
for dist in data_dist_click.Dist_Name.unique():
    df_temp = data_dist_click[data_dist_click['Dist_Name']==dist]
    model = KMeans(n_clusters=20, random_state=0)
    model.fit_predict(df_temp[['x', 'y']])
    df_temp['label'] = model.labels_
    max_label = df_temp.groupby('label').count().idxmax()[0]
    points = df_temp[df_temp['label']==max_label]['geometry'].tolist()
    list_points+=points

C:\Users\Dang\anaconda3\envs\CS116\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\Dang\anaconda3\envs\CS116\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\Dang\anaconda3\envs\CS116\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [15]:
#model = KMeans(20)

In [16]:
#model.fit(data_dist_click[["Dist_ID","dbh","x","y"]].values)

In [17]:
#data_dist_click["label"] = model.labels_

In [18]:
#d4hm = data_dist_click[["geometry","Dist_Name","label"]]

In [19]:
def get_max_label_for_dist(dataframe, name_dist):
    return dataframe[dataframe["Dist_Name"]==name_dist][["geometry","label"]].groupby("label").count().idxmax()[0]    

In [20]:
def get_max_label_for_map(dataframe):
    new_data = {"Dist_Name":[],"label":[]}
    for name_dist in dataframe["Dist_Name"].unique():
        new_data["Dist_Name"].append(name_dist)
        new_data["label"].append(get_max_label_for_dist(dataframe, name_dist))
    return pd.DataFrame(new_data)

In [21]:
#Ml4m=get_max_label_for_map(d4hm)

In [22]:
#points = data_dist_click[data_dist_click["Dist_Name"].isin(Ml4m["Dist_Name"])&data_dist_click["label"].isin(Ml4m["label"])]

In [23]:
from folium import plugins
m = folium.Map( location=[10.772095,106.701965], zoom_start=12, tiles='OpenStreetMap')
# for _, r in data_new.iterrows():
#     # Without simplifying the representation of each borough,
#     # the map might not be displayed
#     sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
#     geo_j = sim_geo.to_json()
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: {'fillColor': 'orange'})
#     folium.Popup(r['Dist_Name']).add_to(geo_j)
#     geo_j.add_to(m)

#heat_data = [[point.xy[1][0], point.xy[0][0]] for point in points["geometry"] ]
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in list_points]

heat_data
plugins.HeatMap(heat_data).add_to(m)

m

In [24]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import time

In [25]:
delay=5
fn=r'./output/heatmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

browser = webdriver.Chrome(ChromeDriverManager().install())

In [26]:
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot(r'./output/heatmap.png')
browser.quit()